In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

DATADIR = "C:/Users/akino/Documents/Deep Learning/sendtex/PetImages"

CATEGORIES = ["Dog", "Cat"]
IMG_SIZE = 50
"""
for category in CATEGORIES:  # do dogs and cats
    path = os.path.join(DATADIR,category)  # create path to dogs and cats
    for img in os.listdir(path):  # iterate over each image per dogs and cats
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
      #  plt.imshow(img_array, cmap='gray')  # graph it
       # plt.show()  # display!

        break  # we just want one for now so break
    break

#So that's a 375 tall, 500 wide, and 3-channel image. 3-channel is because it's RGB (color). We definitely don't 
#want the images that big, but also various images are different shapes, and this is also a problem

IMG_SIZE = 50

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.imshow(new_array, cmap='gray')
plt.show()
"""
training_data = []
def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

create_training_data()
len(training_data)

100%|██████████| 12501/12501 [01:05<00:00, 191.61it/s]


24946

In [2]:
#shuffle the data

import random

random.shuffle(training_data)
X = []
y = []

In [3]:
for features,label in training_data:
    X.append(features)
    y.append(label)

#print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [7]:
#----------------build the model---------------------------------------


import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))#skips the -1
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 2D feature maps to 1D feature vectors
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#model.fit(X, y, epochs=10, batch_size=30, verbose = 1)
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2,
restore_best_weights=True,
verbose=1)

history = model.fit(X, y,
batch_size=32,
callbacks=[es_callback],
validation_split=0.1, epochs=10,verbose=1)


Epoch 1/10
702/702 [==============================] - 245s 348ms/step - loss: 0.6485 - accuracy: 0.6258 - val_loss: 0.6101 - val_accuracy: 0.6890
Epoch 2/10
702/702 [==============================] - 265s 378ms/step - loss: 0.5910 - accuracy: 0.6922 - val_loss: 0.5708 - val_accuracy: 0.7102
Epoch 3/10
702/702 [==============================] - 264s 376ms/step - loss: 0.5575 - accuracy: 0.7179 - val_loss: 0.5354 - val_accuracy: 0.7351
Epoch 4/10
702/702 [==============================] - 263s 374ms/step - loss: 0.5122 - accuracy: 0.7498 - val_loss: 0.5041 - val_accuracy: 0.7607
Epoch 5/10
702/702 [==============================] - 261s 373ms/step - loss: 0.4839 - accuracy: 0.7686 - val_loss: 0.4853 - val_accuracy: 0.7743
Epoch 6/10
702/702 [==============================] - 262s 374ms/step - loss: 0.4572 - accuracy: 0.7860 - val_loss: 0.5137 - val_accuracy: 0.7539
Epoch 7/10
702/702 [==============================] - 259s 369ms/step - loss: 0.4414 - accuracy: 0.7940 - val_loss: 0.4572 -

In [8]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 48, 48, 256)       2560      
_________________________________________________________________
activation_9 (Activation)    (None, 48, 48, 256)       0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 24, 24, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 22, 22, 256)       590080    
_________________________________________________________________
activation_10 (Activation)   (None, 22, 22, 256)       0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 30976)            